In [25]:
1+1

2

In [26]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [27]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [28]:
wiki.name

'wikipedia'

In [29]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [30]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model = "llama3.2"))
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f792c75bef0>, search_kwargs={})

In [31]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [32]:
retriever_tool.name

'langsmith_search'

In [33]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

## arxiv and wikipedia are inbuilt tools provided by langchain , if one want to make a custom tool follow second approch as shown above

In [34]:
tools=[wiki,arxiv,retriever_tool]

In [35]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/user/Bhaskar 2.0/Langchain/venv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7f79631f6840>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_com

In [36]:
##The core idea of agents is to use a language
#  model to choose a sequence of actions to take. 
# In chains, a sequence of actions is hardcoded (in code).
#  In agents, a language model is used as a reasoning 
# engine to determine which actions to take and in which order.

In [37]:
from langchain_ollama import OllamaLLM

# Initialize the open-source LLM
llm = OllamaLLM(model="llama3.2", temperature=0)

### Agents


In [39]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [40]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [ ]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools ,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain_ollama import OllamaLLM

# Define your LLM
llm = OllamaLLM(model="llama3.2", temperature=0)

# Define tools (e.g., wiki, arxiv, retriever_tool)
tools = [
    Tool(name="wiki", func=wiki.run, description="Access Wikipedia information."),
    Tool(name="arxiv", func=arxiv.run, description="Access Arxiv papers."),
    Tool(name="retriever_tool", func=retriever_tool.run, description="Access stored documents."),
]

# Initialize the agent
agent_executor = initialize_agent(tools, 
                                  llm, 
                                  agent="zero-shot-react-description", 
                                  handle_parsing_errors=True,
                                  verbose=True)


In [48]:
response = agent_executor.run("Tell me about Langsmith")
print(response)




> Entering new AgentExecutor chain...
Question: Tell me about Langsmith
Thought: Langsmith is a language model that was developed by Google. It's designed to generate human-like text based on a given prompt or topic.
Action: wiki
Action Input: 'Langsmith'
Observation: No good Wikipedia Search Result was found
Thought:Question: Tell me about Langsmith
Thought: Langsmith is a language model that was developed by Google. It's designed to generate human-like text based on a given prompt or topic.
Action: arxiv
Action Input: 'Langsmith'
Observation: Published: 2020-10-09
Title: Langsmith: An Interactive Academic Text Revision System
Authors: Takumi Ito, Tatsuki Kuribayashi, Masatoshi Hidaka, Jun Suzuki, Kentaro Inui
Summary: Despite the current d
Thought:Question: Tell me about Langsmith
Thought: Langsmith is a language model that was developed by Google. It's designed to generate human-like text based on a given prompt or topic.
Action: arxiv
Action Input: 'Langsmith'
Observation: Publis

In [49]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...
Action: arxiv
Action Input: {'query': '1605.08386'}
Observation: No good Arxiv Result was found
Thought:Question: What's the paper 1605.08386 about?
Thought: Action: arxiv
Action Input: {'query': '1605.08386'}
Observation: No good Arxiv Result was found
Thought:Question: What's the paper 1605.08386 about?
Thought: Action: arxiv
Action Input: {'query': '1605.08386'}
Observation: No good Arxiv Result was found
Thought:Question: What's the paper 1605.08386 about?
Thought: Action: arxiv
Action Input: {'query': '1605.08386'}
Observation: No good Arxiv Result was found
Thought:Question: What's the paper 1605.08386 about?
Thought: Action: arxiv
Action Input: {'query': '1605.08386'}
Observation: No good Arxiv Result was found
Thought:Question: What's the paper 1605.08386 about?
Thought: Action: arxiv
Action Input: {'query': '1605.08386'}
Observation: No good Arxiv Result was found
Thought:Question: What's the paper 1605.08386 about?
Thought: Action: arxi

{'input': "What's the paper 1605.08386 about?",
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [52]:
res=agent_executor.run({"input": "who is pickachu?"})
print(res)




> Entering new AgentExecutor chain...
Question: who is Pikachu?
Thought: I need to access a reliable source of information about Pokémon.
Action: wiki
Action Input: {"query": "Pikachu"}
Observation: Page: Timeline of computer viruses and worms
Summary: This timeline of computer viruses and worms presents a chronological timeline of noteworthy computer viruses, computer worms, Trojan horses, similar malware, related research and events.
Thought:Question: who is Pikachu?
Thought: Question: who is Pikachu?
Thought: I need to access a reliable source of information about Pokémon.
Action: wiki
Action Input: {"query": "Pikachu"}
Observation: Page: Timeline of computer viruses and worms
Summary: This timeline of computer viruses and worms presents a chronological timeline of noteworthy computer viruses, computer worms, Trojan horses, similar malware, related research and events.
Thought:Question: who is Pikachu?
Thought: I need to access a reliable source of information about Pokémon.
Actio

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I can answer questions in the format you specified. Go ahead and ask your question.

(Note: I'll assume that the tool_input is a dictionary with a "query" key containing the search query, and the verbose, start_color, color, callbacks, tags, metadata, run_name, run_id, config, and tool_call_id parameters are not relevant for this specific action.)`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [53]:
input_data = {"input": "Who is Pikachu?"}
response = agent_executor.run(input_data)




> Entering new AgentExecutor chain...
Question: Who is Pikachu?
Thought: I need to access a reliable source of information about Pikachu.
Action: wiki
Action Input: "Pikachu"
Observation: Page: Pikachu
Summary: Pikachu ( ; Japanese: ピカチュウ, Hepburn: Pikachū) is a Pokémon species in Nintendo and Game Freak's Pokémon media franchise, and the franchise's mascot. First introduced in the video games Pokémon Red and Blue, it was created by A
Thought:Question: Who is Pikachu?
Thought: Question: Who is Pikachu?
Thought: I need to access a reliable source of information about Pikachu.
Action: wiki
Action Input: "Pikachu"
Observation: Page: Pikachu
Summary: Pikachu ( ; Japanese: ピカチュウ, Hepburn: Pikachū) is a Pokémon species in Nintendo and Game Freak's Pokémon media franchise, and the franchise's mascot. First introduced in the video games Pokémon Red and Blue, it was created by A
Thought:I'll answer your questions using the provided format.

Question: Who is Pikachu?
Thought: I need to access 